In [ ]:
%matplotlib notebook

In [ ]:
from androguard.misc import AnalyzeAPK
from androguard.session import Save, Session, Load
from androguard.core.bytecode import FormatClassToJava
from androguard.core.analysis.analysis import StringAnalysis

import networkx as nx
import matplotlib.pyplot as plt

from os import path
from collections import defaultdict

import re

In [ ]:
AG_SESSION_FILE = "./Androguard.ag"

In [ ]:
def load_androguard(force_reload=False, write_session=True):
    if (not force_reload) and path.exists(AG_SESSION_FILE):
        print("Loading Existing Session")
        s = Load(AG_SESSION_FILE)
    else:
        print("Loading Session from Apk")
        s = Session()
        a, d, dx = AnalyzeAPK(
            "../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
            , s)
        if write_session:
            print("Saving Loaded Session to", AG_SESSION_FILE)
            Save(s, AG_SESSION_FILE)
        return a, d, dx

# Saving Session causes Kernel disconnection, Loading it causes a EOF Error
a, d, dx = load_androguard(force_reload=True, write_session=False)

In [ ]:
type_descriptors = {
    "void": "V",
    "boolean": "Z",
    "byte": "B",
    "short": "S",
    "char": "C",
    "int": "I",
    "long": "J",
    "float": "F",
    "double": "D"
}

def get_as_type_descriptor(arg):
    if arg.endswith("[]"):
        return "[" + get_as_type_descriptor(arg[:-2])
    if arg in type_descriptors:
        return type_descriptors[arg]
    return FormatClassToJava(arg)

In [ ]:
def strip_return(name):
    # re_strip_return = re.compile(r"\((.*)\)")
    # return re_strip_return.search(name).group(1)
    return name[1:name.index(")")]

In [ ]:
tests_1 = ["java.lang.String", "java.lang.String[]", "void", "int[]", "char", "java.lang.Object[][]"]
for val in tests_1:
    print(val, '-', get_as_type_descriptor(val))

tests_2 = ["(I)I", "(C)Z", "(Ljava/lang/CharSequence; I)I"]
for val in tests_2:
    print(val, '-', strip_return(val))

In [ ]:
class MethodDec:
    def __init__(self, name, *param_types):
        self.name = name
        self.param_types = param_types
    
    def get_method_parameter_types(self):
        li = []
        return list(map(get_as_type_descriptor, self.param_types))

In [ ]:
decs_to_find = {
    "rD5": MethodDec("a", "rD5", "qD5")
}

resolved_classes = [dx.get_class_analysis(get_as_type_descriptor(k)) for k, _ in decs_to_find.items()]

In [ ]:
resolved_methods = []
for (clazz_name, method_dec), class_analysis in zip(decs_to_find.items(), resolved_classes):
    for method in class_analysis.get_methods():
        if method_dec.name == method.name:
            m_dec_types = " ".join(method_dec.get_method_parameter_types())
            if m_dec_types == strip_return(str(method.get_descriptor())):
                print("Found Class and Method", f"{clazz_name}#{method_dec.name}({m_dec_types})")
                resolved_methods.append(method)

In [ ]:
m_dict, c_dict = defaultdict(set), defaultdict(set)
for s in dx.get_strings():
    xrefs = s.get_xref_from()
    # if len(xrefs) > 2202:
    # continue
    for x in xrefs:
        c_ref, m_ref = x
        for r_m in resolved_methods:
            if m_ref == r_m:
                # print(len(xrefs), s.value)
                m_dict[r_m].add(s.value)
            elif c_ref.name == r_m.class_name:
                # TODO: Add String Finding for Classes
                pass

print(m_dict)

In [ ]:
def flat_map(f, li): return (f(y) for x in li for y in x)

# str_dic = {s: s.get_xref_from() for s in dx.strings}
for s in dx.get_strings():
    for k, v in m_dict.items():
        if s.value in v:
            for x in s.get_xref_from():
                m_ref = x[1]
                print("Possible Candidate: ", r_m)